# Clasificación con conjunto de datos _iris_

[**Juan Gómez Romero**](https://decsai.ugr.es/~jgomez)  
Departamento de Ciencias de la Computación e Inteligencia Artificial  
Universidad de Granada  
This work is licensed under the [GNU General Public License v3.0](https://choosealicense.com/licenses/gpl-3.0/).

---

### Activar GPU
_Entorno de ejecución > Cambiar tipo de entorno de ejecución_

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Cargar datos de _iris_
Cargamos los datos de [ _iris_](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) incluidos en el paquete [`scikit-learn`](https://scikit-learn.org/).
- features: entrada (150 x 4)
- labels: salida (150 x 1)

In [ ]:
from sklearn import datasets
data = datasets.load_iris()

features = data.data[:, :]
labels   = data.target

print(data['DESCR'])

Visualizamos los datos. Las clases de _iris_ son fácilmente separables con las variables `petal_length` y `petal_width`.

In [ ]:
import matplotlib.pyplot as plt
plt.title('Iris dataset ')
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.scatter(features[:, 0], features[:, 1], c = labels)
plt.show()

plt.title('Iris dataset')
plt.xlabel('Petal length')
plt.ylabel('Petal width')
plt.scatter(features[:, 2], features[:, 3], c = labels)
plt.show()

## Definir arquitectura
Definimos la arquitectura de la red neuronal:
- Entrada: 4 neuronas
- Oculta: 2 x 16 neuronas (*sigmoid*)
- Salida: 3 neuronas (*softmax*)

<img src="https://github.com/jgromero/eci2019-DRL/blob/master/Tema%202%20-%20Aprendizaje%20Profundo/code/iris-nn.png?raw=true" width="500">

In [ ]:
import torch.nn as nn

torch.manual_seed(1)

net = nn.Sequential(
    nn.Linear(in_features = features.shape[1], out_features = 16),
    nn.ReLU(),
    nn.Linear(in_features = 16, out_features = 16),
    nn.ReLU(),
    nn.Linear(in_features = 16, out_features = 3),
    nn.Softmax(dim = 0)
)
net = net.to(device)

net

Calcular salida para una entrada del conjunto del entrenamiento.

In [ ]:
input = torch.tensor(features[0, :], dtype=torch.float32, device=device)
output = net(input)
print("Predicción: " , output)
print("Predicción clase: " , torch.argmax(output))
print("Salida esperada: ", labels[0])

## Entrenar red

Preparamos los datos al formato esperado por la red y separamos los conjuntos de entrenamiento y test.

In [ ]:
import torch.utils.data

x_dataset = torch.tensor(features, dtype = torch.float32)
y_dataset = torch.tensor(labels, dtype = torch.long)
dataset   = torch.utils.data.TensorDataset(x_dataset, y_dataset)

perc_split = 0.8
n_train = round(perc_split * len(features))
n_val   = len(features) - n_train
train_set, val_set = torch.utils.data.random_split(dataset, [n_train, n_val])

Definimos la función de pérdida y el algoritmo de optimización que queremos utilizar sobre los parámetros de la red.

In [ ]:
import torch.optim

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

Lanzamos el proceso de entrenamiento.

In [ ]:
batch_size = 50
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

n_epochs = 1000

for epoch in range(n_epochs):  
    epoch_loss = 0.0
    
    for (i, data) in enumerate(train_loader, 0):
        x, y = data
    
        x = x.to(device)
        y = y.to(device)
    
        optimizer.zero_grad()       # reset gradientes
                
        y_hat = net(x)              # calcular salida forward
        loss = criterion(y_hat, y)  # calcular pérdida
        loss.backward()             # propagar error hacia atrás
        optimizer.step()            # modificar pesos
    
        epoch_loss += loss.item()        
    
    if(epoch % 100 == 0):
        print("Epoch [%4d/%4d], epoch loss: %.3f" % (epoch, n_epochs, float(epoch_loss) / batch_size))

## Validación
Calculamos la precisión del modelo sobre los datos de validación.

In [ ]:
val_loader = torch.utils.data.DataLoader(val_set, batch_size=100, shuffle=False)

correct = 0
total = 0

with torch.no_grad():        # no se calculan gradientes, solo paso forward
    for x, y in val_loader:
        x = x.to(device)
        y = y.to(device)
    
        y_hat = net(x)
        _, predicted = torch.max(y_hat, dim = 1)    
        total = total + y.shape[0]
        correct += int((predicted == y).sum())

    print("Precisión (validación): %.2f" % (correct / total))

Podemos seguir variando los valores iniciales de los pesos, los hiperparámetros y el proceso de entrenamiento para conseguir valores de precisión mayores:
- `torch.manual_seed`
- `batch_size`
- `n_epochs`
- `nn.CrossEntropyLoss`
- `torch.optim.SGD`

**Atención**: Esto implica utilizar los datos validación para tomar decisiones de mejora la red, por lo que ya no se podría considerar un conjunto independiente. En estos casos, suele reservarse un conjunto de datos adicional, denominado _test_.

---

### EJERCICIO

¿Podrías extender este código para otro problema de clasificación de los incluidos en 
en el paquete [`scikit-learn`](https://scikit-learn.org/). Por ejemplo, [cancer](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html).


---